# Poland 2D Line 001 apply acquisition geometry

Using the "dropped auxilliary channels" from a previous example and the positioning files, this example illustrates how to extract values from the dataframe and apply to headers in segy.

In [1]:
import os
import math
from pathlib import Path
from poland2d_context import segytools

from segytools.segy_trace_header import COORDINATE_SCALAR_MULTIPLIER

from positioning.sps_parser import SPS
from positioning.rps_parser import RPS
from positioning.xps_parser import XPS

def midpoint_coordinates(sx,sy,gx,gy):
    mx = (float(sx) + float(gx)) / 2.0
    my = (float(sy) + float(gy)) / 2.0
    return mx, my

def euclidean_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx**2 + dy**2)

def azimuth_degrees(sx, sy, gx, gy):
    # TODO: fix so that the azimuth is always source to receiver; atan2 {-PI, PI}
    return (180.0 / math.pi) * math.atan2(sx - gx, sy - gy)

sps = SPS(f="data/Line_001.SPS")
sps_df = sps.dataframe()

rps = RPS(f="data/Line_001.RPS")
rps_df = rps.dataframe()

xps = XPS(f="data/Line_001.XPS")
xps_df = xps.dataframe()

In [2]:
TEXTENCODE = 'ebcdic'
ENDIANESS = '>'
TEXTHEADERLENGTH = 3200
FILEHEADERLENGTH = 400
TRCHEADERLENGTH = 240

segyfile = "data/Line_001_NO_AUX.sgy"
pathsegyfile = Path(str(segyfile))
assert (pathsegyfile.is_file())

segyfilesize = os.path.getsize(segyfile)

# Initialize an empty bytearray that is the same size as the input segy file.
# This is only possible with small files where segyfilesize < available memory.
b_sgy_arr = bytearray(segyfilesize)

file_header = segytools.SegyFileHeaderRev2()
trace_header = segytools.SegyTraceHeaderRev2()

# 'rb' is "read bytes"
with open(segyfile, 'rb') as fobj:
    # read the first 3200 bytes.
    # This will always be 3200 byte textual file header
    b_text_header = fobj.read(TEXTHEADERLENGTH)
    b_sgy_arr[0:TEXTHEADERLENGTH] = b_text_header
    last_byte = fobj.tell()

    b_file_header = fobj.read(FILEHEADERLENGTH)
    file_header.set_header_values(buf=b_file_header, endianess=ENDIANESS)
    # modify file header here ...
    b_sgy_arr[last_byte:last_byte+FILEHEADERLENGTH] = file_header.to_bytes(endianess=ENDIANESS)
    last_byte = fobj.tell()
    
    sample_size_in_bytes = file_header.sample_format_size_in_bytes()
    trc_data_length_in_bytes = file_header.num_samples_per_trace.value * sample_size_in_bytes
    
    # Loop through traces ...
    while fobj.tell() < segyfilesize:
        # TRACE HEADER
        b_trace_header = fobj.read(TRCHEADERLENGTH)
        trace_header.set_header_values(buf=b_trace_header, endianess=ENDIANESS)
        # fix incorrect coordinate scalars
        trace_header.xy_scalar.value = int(-100)
        trace_header.z_scalar.value = int(1)
        # apply acquisition geometry to headers
        trace_header.shotpoint.value = xps.source_point(ffid=trace_header.field_record_number.value)
        # scale x,y coords and cast to int before assigning to header value; NOTE That the xy scalers in the headers are not correct
        src_x = round(sps.source_x(source_point=trace_header.shotpoint.value) / trace_header.xy_scalar.mapped_value)
        #--print(sps.source_x(source_point=trace_header.shotpoint.value))
        #--print(trace_header.xy_scalar.value, trace_header.xy_scalar.mapped_value)
        #--print(src_x)
        trace_header.src_x_coord.value = int(src_x)
        src_y = round(sps.source_y(source_point=trace_header.shotpoint.value) / trace_header.xy_scalar.mapped_value)
        trace_header.src_y_coord.value = int(src_y)
        src_z = round(sps.source_elevation(source_point=trace_header.shotpoint.value))
        trace_header.src_elevation.value = int(src_z)
        src_stat = sps.source_static(source_point=trace_header.shotpoint.value)  # assuming milliseconds in SPS file
        trace_header.src_static_correction.value = int(src_stat)
        #--print(trace_header.trc_num_within_field_record.value)
        trace_header.group_num.value = xps.reciever_station(ffid=trace_header.field_record_number.value, chan=trace_header.trc_num_within_field_record.value)
        rcv_x = round(rps.receiver_x(receiver_point=trace_header.group_num.value) / trace_header.xy_scalar.mapped_value)
        trace_header.rcv_x_coord.value = int(rcv_x)
        rcv_y = round(rps.receiver_y(receiver_point=trace_header.group_num.value) / trace_header.xy_scalar.mapped_value)
        trace_header.rcv_y_coord.value = int(rcv_y)
        rcv_z = round(rps.receiver_elevation(receiver_point=trace_header.group_num.value))
        trace_header.rcv_elevation.value = int(rcv_z)
        rcv_stat = round(rps.receiver_static(receiver_point=trace_header.group_num.value))
        trace_header.rcv_static_correction.value = int(rcv_stat)
        # calculate the midpoint coordinates and offset
        cmpx, cmpy = midpoint_coordinates(sx=trace_header.src_x_coord.value * 0.01, 
                                          sy=trace_header.src_y_coord.value * 0.01, 
                                          gx=trace_header.rcv_x_coord.value * 0.01, 
                                          gy=trace_header.rcv_y_coord.value * 0.01)
        trace_header.ens_x_coord.value = int(round(cmpx * 100))
        trace_header.ens_y_coord.value = int(round(cmpy * 100))
        offset = euclidean_distance(x1=trace_header.src_x_coord.value * 0.01, 
                                    y1=trace_header.src_y_coord.value * 0.01, 
                                    x2=trace_header.rcv_x_coord.value * 0.01, 
                                    y2=trace_header.rcv_y_coord.value * 0.01)
        trace_header.offset.value = int(round(offset))
        
        # write trace header to bytearray `b_sgy_arr`
        b_sgy_arr[last_byte:last_byte+TRCHEADERLENGTH] = trace_header.to_bytes(endianess=ENDIANESS)
        last_byte += TRCHEADERLENGTH

        # TRACE DATA
        if trace_header.num_samples.value != file_header.num_samples_per_trace.value:
            trc_data_length_in_bytes = file_header.num_samples_per_trace.value * sample_size_in_bytes
        b_trace_data = fobj.read(trc_data_length_in_bytes)
        b_sgy_arr[last_byte:last_byte+trc_data_length_in_bytes] = b_trace_data
        last_byte += trc_data_length_in_bytes

    fobj.close()

# write output segy; use only with small segy files
segy_file_out = os.path.join(os.getcwd(), "data", "Line_001_NO_AUX_geom.sgy")
with open(segy_file_out, "wb") as fobj2:
    bsgyout = bytes(b_sgy_arr[:last_byte])
    fobj2.write(bsgyout)
    fobj2.close() 